In [1]:
from itertools import product
from mip import Model, xsum, minimize, BINARY

import os
import pandas as pd
import re

import datetime
from datetime import timedelta, date #for later datetimes processing

import matplotlib.pyplot as plt #plotting
import plotly.express as px #generate beautiful interactive plots

In [32]:
os.chdir('/Users/chenya68/Documents/GitHub/uo-mip')
#df_raw = pd.read_excel('data/OSD_schedules.xlsx',sheet_name = 4) #sheet_name = 2
df_raw = pd.read_excel('data/OSD_current_schedule.xlsx',sheet_name = 'draft1_list-1204') 
df_raw.columns = [re.sub('[^A-Za-z0-9Δ]+', '_', element) for element in df_raw.columns]
df_raw.fillna(0, inplace=True)
df_raw

,task_index,task_name,processing_time_weeks_,engineers,rooms,LM_40_VBE_1_BN_,LM_40_VBE_2_BN_,LM_40_Isolator_BN_,PTM_300_BN_,Sieve,...,1_Mike,2_Sergio,3_Rob,4_Syed,5_Joe,6_James_T,7_Liam,8_Sadia,9_Gabriella,10_Jan
0,0,start-0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,MK-8591-C,7,2,1,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,MK-1167-B,4,2,1,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,MK-3543-1-B,4,2,1,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,MK-3543-TroubleShooting-B,1,2,1,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,5,MK-3543-2-B,1,2,1,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,6,MK-3543-Requal-B,1,2,1,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,7,MK-3543-3-B,2,2,1,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,8,MK-3543-Cleaning-B,1,2,1,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,9,MK-1084GMP-D,4,2,1,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [33]:
activities = df_raw[['task_index','task_name']].set_index('task_index').T.to_dict('records')[0]
activities

{0: 'start-0',
 1: 'MK-8591-C',
 2: 'MK-1167-B',
 3: 'MK-3543-1-B',
 4: 'MK-3543-TroubleShooting-B',
 5: 'MK-3543-2-B',
 6: 'MK-3543-Requal-B',
 7: 'MK-3543-3-B',
 8: 'MK-3543-Cleaning-B',
 9: 'MK-1084GMP-D',
 10: 'MK-1084Dev-D',
 11: 'MK-1084Setup-A1',
 12: 'MK-1084-A1',
 13: 'MK-1084-2-D',
 14: 'EP-200-E',
 15: 'Arrival-F',
 16: 'Installation-F',
 17: 'SAT-F',
 18: 'Setup-G',
 19: 'WaterTesting-G',
 20: 'Cleaning-G',
 21: 'Dev-G',
 22: 'Reset-G',
 23: 'GMP-G',
 24: 'end-0'}

In [34]:
n = len(df_raw) - 2

p = df_raw['processing_time_weeks_'].to_list() #processing time of each task

#u = df_raw.iloc[:,4:12].to_numpy().tolist()
#df_raw.iloc[:,4:12]

r_end_col = 31#last column index of equipment

u = df_raw.iloc[:,4:r_end_col].to_numpy().tolist() #rooms and equipment 4:14
df_raw.iloc[:,4:r_end_col]


,rooms,LM_40_VBE_1_BN_,LM_40_VBE_2_BN_,LM_40_Isolator_BN_,PTM_300_BN_,Sieve,Comill,Hammermill,RC_,RC_2,...,Short_ICAPS,Sade_Weight_Sorter,Sejong_Weight_Sorter_2,Spray_Dryer_,50L_Tank,20_100_L_tanks,HME_,Isolators_,Autoinjector_MRT_,TSG
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
len(u[1])

27

In [36]:
#av = df_raw.iloc[:,12:].to_numpy().tolist() #capability of engineers
#df_raw.iloc[:,12:]

av = df_raw.iloc[:,r_end_col:].to_numpy().tolist() #capability of engineers
df_raw.iloc[:,r_end_col:]

,0_Patrick,1_Mike,2_Sergio,3_Rob,4_Syed,5_Joe,6_James_T,7_Liam,8_Sadia,9_Gabriella,10_Jan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [53]:
#c = [11,11,3,1,1,2,2,1,1] #availability of renewable resources: engineers, rooms,...
#c = [11,3,1,1,2,2,1,1] #availability of renewable resources: rooms, LM_40_VBE_1_BN_	LM_40_VBE_2_BN_	LM_40_Isolator_BN_	PTM_300_BN_	Sieve	Spray_Dryer_	Film_Coater_
#c = [11,3,1,1,2,2,1,1,1,1] #availability of renewable resources: rooms, LM_40_VBE_1_BN_	LM_40_VBE_2_BN_	LM_40_Isolator_BN_	PTM_300_BN_	Sieve	Spray_Dryer_	50L_Tank	Film_Coater_	Autoinjector_MRT_
c = [2]*len(u[0])
c[0] = 11
c[1] = 3
c[4] = 2
c[5] = 2
allowed_task = 2 #number of tasks a worker can work on at the same time

required_worker = 2

B = 10000 #big constant

"""
good1
S = [[0, 1], [1,16],
     [0, 2], [2,16],
     [0, 3], [3,4],[4,5],[5,6],[6,7],[7,8],[8,16],
     [0,9], [9,10],[10,11],[11,12],[12,13],[13,16],
     [0,14],[14,16],
     [0,15],[15,16]]
"""

"""
good0
S = [[0, 1], [1,12],
     [0, 2], [2,12],
     [0, 3], [3,4],[4,5],[5,12],
     [0,6], [6,7],[7,8],[8,9],[9,10],[10,12],
     [0,11],[11,12]]

"""


S = [[0, 1], [1,24],
     [0, 2], [2,24],
     [0, 3], [3,4],[4,5],[5,6],[6,7],[7,8],[8,24],
     [0,9], [9,10],[10,13],[13,24],
     [0,11],[11,12],[12,24],
     [0,14],[14,24],
     [0,15],[15,16],[16,17],[17,24],
     [0,18],[18,19],[19,20],[20,21],[21,22],[22,23],[23,24]]

"""
S = [[0, 1], [0, 2], [0, 3], [0,6], [0,9],
     [3,4],[4,5],[6,7],[7,8],
      [1, 10], [2, 10],[5, 10], [8, 10],[9,10]]
"""

(R, J, T, T1, W) = (range(len(c)), range(len(p)), range(sum(p)), range(1,sum(p)), range(len(av[0]))) #Resource set, Task set, Time set, Worker set


In [54]:
model = Model()

#define decision variables
#v_{jt}: if time t is the start time of task j
v = [[model.add_var(name="v({},{})".format(j, t), var_type=BINARY) for t in T] for j in J]

#f_{jt}: if time t is the end time of task j
f = [[model.add_var(name="f({},{})".format(j, t), var_type=BINARY) for t in T] for j in J]

#x_{jt}: if task j is being implemented at time t
x = [[model.add_var(name="x({},{})".format(j, t), var_type=BINARY) for t in T] for j in J]

#z_{jwt}: if task j is operated by worker w at time t
z = [[[model.add_var(name="z({},{},{})".format(j,t,w), var_type=BINARY) for w in W] for t in T] for j in J]

model.objective = xsum(t * v[n + 1][t] for t in T)

for j in J:
    model += xsum(v[j][t] for t in T) == 1
    model += xsum(f[j][t] for t in T) == 1


for j in range(11,13): #task 11 and 12 (process A1) can't be completed during holiday and yearend shutdown
     model += xsum(v[j][t] for t in range(16,20)) == 0
     model += xsum(x[j][t] for t in range(16,20)) == 0


for j in [9,10,13]: #task 9,10,13 (process D) can't be completed during week 12
     model += xsum(v[j][t] for t in [12]) == 0
     model += xsum(x[j][t] for t in [12]) == 0

for (r, t) in product(R, T):
    model += (
        xsum(u[j][r] * v[j][t2] for j in J for t2 in range(max(0, t - p[j] + 1), t + 1))
        <= c[r])

for j in J:
    model += xsum(t * v[j][t] for t in T) + p[j] == xsum(t * f[j][t] for t in T)

for (j, s) in S:
    model += xsum(t * v[s][t] - t * v[j][t] for t in T) >= p[j]

#a person can only work on at most cw tasks per time slot
for (t,w) in product(T,W):
    model += xsum(z[j][t][w] for j in J) <= allowed_task

#tasks can't be allocated to workers that do not have the required skill
for (j,w) in product(J,W):
    model += xsum(z[j][t][w] for t in T) <=B*av[j][w]

#per time slot, per task, there need to be at least two workers operating
for (j,t) in product(J,T):
    model += xsum(z[j][t][w] for w in W)>= required_worker*x[j][t]
    model += xsum(z[j][t][w] for w in W)<= B*x[j][t]

for(j,t) in product([0,len(p)-1],T): #tasks are stopped being implemented once finished
     model += x[j][t] == 0
for j in range(1,len(p)-1):
    model += (x[j][0] ==v[j][0])


for (j,t) in product (J,T1):
        model += x[j][t] - x[j][t-1] == v[j][t] - f[j][t]

In [55]:
model.optimize()

print("Schedule: ")
for (j, t) in product(J, T):
    if v[j][t].x >= 0.99:
        print("Job {}: begins at t={} and finishes at t={}".format(j, t, t+p[j]))
print("Makespan = {}".format(model.objective_value))

Starting solution of the Linear programming relaxation problem using Primal Simplex

Clp0024I Matrix will be packed to eliminate 55252 small elements
Coin0506I Presolve 4747 (-1737) rows, 6488 (-12062) columns and 35194 (-58174) elements
Clp0030I 56 infeas 10.965218, obj 14.322192 - mu 2.5765326e-06, its 105, 6295 interior
Clp1000I sum of infeasibilities 2.70475 - average 0.00056978, 217 fixed columns
Coin0506I Presolve 4672 (-75) rows, 6244 (-244) columns and 32907 (-2287) elements
Clp0006I 0  Obj 14.34993 Primal inf 5.8775803 (1248) Dual inf 1.0080582e+16 (3484)
Clp0029I End of values pass after 6244 iterations
Clp0014I Perturbing problem by 0.001% of 2.6030745 - largest nonzero change 2.9296205e-05 ( 0.0025517872%) - largest zero change 2.9943646e-05
Clp0000I Optimal - objective value 13
Clp0000I Optimal - objective value 13
Coin0511I After Postsolve, objective 13, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 13
Clp0000I Optimal - objective value 13
Clp0000I Optimal - 

In [56]:
for j in range(11,13):
    for t in range(16,20):
        print(x[j][t].x)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [57]:
for (j,t) in product(J, T):
    if f[j][t].x == 1:
        print(j,t,f[j][t].x)

0 0 1.0
1 9 1.0
2 12 1.0
3 4 1.0
4 5 1.0
5 10 1.0
6 11 1.0
7 14 1.0
8 15 1.0
9 4 1.0
10 12 1.0
11 1 1.0
12 15 1.0
13 15 1.0
14 8 1.0
15 1 1.0
16 3 1.0
17 9 1.0
18 1 1.0
19 2 1.0
20 3 1.0
21 5 1.0
22 6 1.0
23 13 1.0
24 15 1.0


In [58]:
for (j,t) in product(J, T):
    if x[j][t].x == 1:
        print(j,t,x[j][t].x)

1 2 1.0
1 3 1.0
1 4 1.0
1 5 1.0
1 6 1.0
1 7 1.0
1 8 1.0
2 8 1.0
2 9 1.0
2 10 1.0
2 11 1.0
3 0 1.0
3 1 1.0
3 2 1.0
3 3 1.0
4 4 1.0
5 9 1.0
6 10 1.0
7 12 1.0
7 13 1.0
8 14 1.0
9 0 1.0
9 1 1.0
9 2 1.0
9 3 1.0
10 4 1.0
10 5 1.0
10 6 1.0
10 7 1.0
10 8 1.0
10 9 1.0
10 10 1.0
10 11 1.0
11 0 1.0
12 11 1.0
12 12 1.0
12 13 1.0
12 14 1.0
13 14 1.0
14 5 1.0
14 6 1.0
14 7 1.0
15 0 1.0
16 2 1.0
17 7 1.0
17 8 1.0
18 0 1.0
19 1 1.0
20 2 1.0
21 4 1.0
22 5 1.0
23 11 1.0
23 12 1.0


In [59]:
for (j,t,w) in product(J, T,W):
    if z[j][t][w].x == 1:
        print('task:',j,'time:',t,'worker:',w,z[j][t][w].x)

task: 1 time: 2 worker: 1 1.0
task: 1 time: 2 worker: 9 1.0
task: 1 time: 3 worker: 1 1.0
task: 1 time: 3 worker: 9 1.0
task: 1 time: 4 worker: 1 1.0
task: 1 time: 4 worker: 9 1.0
task: 1 time: 5 worker: 1 1.0
task: 1 time: 5 worker: 9 1.0
task: 1 time: 6 worker: 1 1.0
task: 1 time: 6 worker: 9 1.0
task: 1 time: 7 worker: 1 1.0
task: 1 time: 7 worker: 9 1.0
task: 1 time: 8 worker: 1 1.0
task: 1 time: 8 worker: 9 1.0
task: 2 time: 8 worker: 0 1.0
task: 2 time: 8 worker: 5 1.0
task: 2 time: 9 worker: 0 1.0
task: 2 time: 9 worker: 5 1.0
task: 2 time: 10 worker: 0 1.0
task: 2 time: 10 worker: 5 1.0
task: 2 time: 11 worker: 0 1.0
task: 2 time: 11 worker: 5 1.0
task: 3 time: 0 worker: 0 1.0
task: 3 time: 0 worker: 5 1.0
task: 3 time: 1 worker: 0 1.0
task: 3 time: 1 worker: 5 1.0
task: 3 time: 2 worker: 0 1.0
task: 3 time: 2 worker: 5 1.0
task: 3 time: 3 worker: 0 1.0
task: 3 time: 3 worker: 5 1.0
task: 4 time: 4 worker: 0 1.0
task: 4 time: 4 worker: 5 1.0
task: 5 time: 9 worker: 0 1.0
task: 

In [60]:
SCHEDULE = {}
for (j, t) in product(J, T):
  if v[j][t].x >= 0.99: #only get the x values == 1
     a = activities[j]
     SCHEDULE[a] = {'s':t,'f':t+p[j]}
     print(f'Activity {j}, begins at t={t} and finishes at {t+p[j]}')

out = pd.DataFrame(SCHEDULE).T
SCHEDULE = out.T.to_dict()

Activity 0, begins at t=0 and finishes at 0
Activity 1, begins at t=2 and finishes at 9
Activity 2, begins at t=8 and finishes at 12
Activity 3, begins at t=0 and finishes at 4
Activity 4, begins at t=4 and finishes at 5
Activity 5, begins at t=9 and finishes at 10
Activity 6, begins at t=10 and finishes at 11
Activity 7, begins at t=12 and finishes at 14
Activity 8, begins at t=14 and finishes at 15
Activity 9, begins at t=0 and finishes at 4
Activity 10, begins at t=4 and finishes at 12
Activity 11, begins at t=0 and finishes at 1
Activity 12, begins at t=11 and finishes at 15
Activity 13, begins at t=14 and finishes at 15
Activity 14, begins at t=5 and finishes at 8
Activity 15, begins at t=0 and finishes at 1
Activity 16, begins at t=2 and finishes at 3
Activity 17, begins at t=7 and finishes at 9
Activity 18, begins at t=0 and finishes at 1
Activity 19, begins at t=1 and finishes at 2
Activity 20, begins at t=2 and finishes at 3
Activity 21, begins at t=4 and finishes at 5
Activit

In [61]:
start_date = datetime.date(2023, 9, 3)
#start_date = datetime.date.today()

SCHEDULE_DF = pd.DataFrame(SCHEDULE).T
SCHEDULE_DF['Start_Date'] = [str(start_date + datetime.timedelta(weeks=s)) for s in SCHEDULE_DF['s']]
SCHEDULE_DF['Finish_Date'] =[str(start_date + datetime.timedelta(weeks=f)) for f in SCHEDULE_DF['f']]

SCHEDULE_DF.to_csv('result/output_schedule-24a-1204.csv') #export csv file
SCHEDULE_DF.to_excel('result/output_schedule-24a-1204.xlsx') #export excel file

In [62]:
from pandas import factorize
SCHEDULE_DF.reset_index(inplace = True)
SCHEDULE_DF.rename(columns={'index': 'task_name'}, inplace=True)
SCHEDULE_DF['task_type'] = SCHEDULE_DF['task_name'].str[-2:]
labels, categories = factorize(SCHEDULE_DF['task_type'])
SCHEDULE_DF["task_label"] = labels
#SCHEDULE_DF['task_name'] = 'T-'+SCHEDULE_DF['task_name']
SCHEDULE_DF

,task_name,s,f,Start_Date,Finish_Date,task_type,task_label
0,start-0,0,0,2023-09-03,2023-09-03,-0,0
1,MK-8591-C,2,9,2023-09-17,2023-11-05,-C,1
2,MK-1167-B,8,12,2023-10-29,2023-11-26,-B,2
3,MK-3543-1-B,0,4,2023-09-03,2023-10-01,-B,2
4,MK-3543-TroubleShooting-B,4,5,2023-10-01,2023-10-08,-B,2
5,MK-3543-2-B,9,10,2023-11-05,2023-11-12,-B,2
6,MK-3543-Requal-B,10,11,2023-11-12,2023-11-19,-B,2
7,MK-3543-3-B,12,14,2023-11-26,2023-12-10,-B,2
8,MK-3543-Cleaning-B,14,15,2023-12-10,2023-12-17,-B,2
9,MK-1084GMP-D,0,4,2023-09-03,2023-10-01,-D,3


In [63]:
import numpy as np
def rgb(): return 'rgb(' + ','.join(map(str, np.random.randint(0, 255, size=3))) + ')'

df = SCHEDULE_DF
df['order'] = df.index

#discrete_map_resource = {r: rgb() for r in df.Resource}
#discrete_sequence_resource = [rgb() for resource in df.Resource.unique()]
discrete_map_task = [rgb() for task in df.task_label]

fig = px.timeline(df, 
                  x_start="Start_Date", 
                  x_end="Finish_Date",
                  #color = 'task_type',
                  #color_discrete_map=discrete_map_task,
                  #category_orders = dict(group = list(activities.keys())),
                  y=df.task_name,
                  title='Optimized Project Gantt Chart',
                  width=800, height=600)
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
#fig.update_layout(yaxis = {"categoryorder":"array", "categoryarray": df['order']})
#fig.update_layout(yaxis_categoryorder = 'total ascending')
fig.update_layout(yaxis={'dtick':1})

"""
fig.update_layout(#margin_pad = 0,
                  yaxis=dict(automargin = True),
                  yaxis_showticksuffix = 'last',
                  yaxis_anchor="free",
                  yaxis_side="right",
                  yaxis_shift=-50,
                  yaxis_title = None,
                 )
"""
#fig.write_html("Gantt_chart-v3.html")
fig.show()

In [30]:
#os.chdir('/Users/chenya68/Documents/GitHub/uo-mip')
SCHEDULE_DF_current =pd.read_excel('result/output_schedule-current.xlsx',sheet_name = 0)
SCHEDULE_DF_current

,Unnamed: 0,s,f,Start_Date,Finish_Date
0,start,0.0,0.0,2023-09-03,2023-09-03
1,MK-8591-C,2.0,9.0,2023-10-01,2023-11-19
2,MK-1167-B,9.0,13.0,2023-11-02,2023-12-10
3,MK-3543-1-B,1.0,5.0,2023-12-10,2024-01-08
4,MK-3543-TroubleShooting-B,NaN,NaN,2024-01-08,2024-01-15
5,MK-3543-2-B,14.0,15.0,2024-01-15,2024-01-22
6,MK-3543-Requal-B,NaN,NaN,2024-01-22,2024-01-29
7,MK-3543-3-B,15.0,17.0,2024-01-29,2024-02-12
8,MK-3543-Cleaning-B,NaN,NaN,2024-02-12,2024-02-19
9,MK-1084GMP-D,0.0,4.0,2023-09-03,2023-10-01


In [31]:
fig = px.timeline(SCHEDULE_DF_current, 
                  x_start="Start_Date", 
                  x_end="Finish_Date", 
                  y=SCHEDULE_DF.task_name,title='Current Project Gantt Chart')
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
#fig.write_html("Gantt_chart-current.html")
fig.update_layout(yaxis={'dtick':1})
fig.show()